<a href="https://colab.research.google.com/github/wma-spu/class-materials/blob/main/DS610_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install java libs and spark.
!apt update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
# set environment variables for java and spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
# add pyspark to sys.path
import findspark
findspark.init()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
dataFrame = spark.read.format("CSV").option("header","true").load('/content/drive/MyDrive/AirlineInfo/*.csv.bz2')
dataFrame.show(5)
dataFrame.printSchema()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    1|         1|        1|   1232|      1225|   1341|      1340|           WN|     2891

In [6]:
dataFrame = dataFrame.withColumn("ArrDelay",dataFrame.ArrDelay.cast('integer')) \
                     .withColumn("DepDelay",dataFrame.DepDelay.cast('integer')) \
                     .withColumn("Cancelled",dataFrame.Cancelled.cast('integer'))
dataFrame = dataFrame[['UniqueCarrier', 'FlightNum', 'Origin', 'Dest', 'DayofMonth', 'DayOfWeek', 'ArrDelay', 'DepDelay', 'Cancelled']]

In [7]:
dataFrame.show(5)
dataFrame.printSchema()

+-------------+---------+------+----+----------+---------+--------+--------+---------+
|UniqueCarrier|FlightNum|Origin|Dest|DayofMonth|DayOfWeek|ArrDelay|DepDelay|Cancelled|
+-------------+---------+------+----+----------+---------+--------+--------+---------+
|           WN|     2891|   SMF| ONT|         1|        1|       1|       7|        0|
|           WN|      462|   SMF| PDX|         1|        1|       8|      13|        0|
|           WN|     1229|   SMF| PDX|         1|        1|      34|      36|        0|
|           WN|     1355|   SMF| PDX|         1|        1|      26|      30|        0|
|           WN|     2278|   SMF| PDX|         1|        1|      -3|       1|        0|
+-------------+---------+------+----+----------+---------+--------+--------+---------+
only showing top 5 rows

root
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DayofMonth: string (nu



**1.   Find the # of flights each airline made so far from 1987 until recent.**





In [25]:
countDF = dataFrame.groupBy(['UniqueCarrier', 'FlightNum']).count()
countDF.show(20)
countDF.persist()

+-------------+---------+-----+
|UniqueCarrier|FlightNum|count|
+-------------+---------+-----+
|           WN|      179| 7401|
|           WN|     1710|10287|
|           WN|      563|16907|
|           WN|      116|10117|
|           WN|      301| 5256|
|           WN|     1293| 8468|
|           WN|     1353| 1002|
|           WN|     1824| 2028|
|           WN|     1635| 5123|
|           WN|      987| 9260|
|           WN|     2292|  764|
|           XE|     3186|  436|
|           XE|     3141|  716|
|           XE|     2856| 1811|
|           XE|     2441| 2242|
|           XE|     2231| 2142|
|           XE|     2509| 1420|
|           YV|     7428|  411|
|           YV|     2991|  131|
|           OH|     5407| 1768|
+-------------+---------+-----+
only showing top 20 rows



DataFrame[UniqueCarrier: string, FlightNum: string, count: bigint]

**2.   Find the mean departure delay per origination airport.**

In [27]:
delayDF = dataFrame.groupBy(['Origin']).mean('DepDelay')
delayDF.show(20)
delayDF_cache = delayDF.cache()

+------+------------------+
|Origin|     avg(DepDelay)|
+------+------------------+
|   BGM| 4.103673055154378|
|   DLG|             10.75|
|   PSE|1.9544658493870404|
|   INL| 6.168539325842697|
|   MSY| 6.554481654634312|
|   GEG| 6.038673098079038|
|   SNA| 5.813108982130751|
|   BUR| 6.695134420234224|
|   GRB| 4.685745396013824|
|   GTF|3.1319266055045873|
|   IDA| 3.023048836147878|
|   GRR| 5.625893559603069|
|   LWB| 7.619393939393939|
|   PVU|               0.0|
|   EUG| 7.953983363108531|
|   PSG|12.997746320163392|
|   PVD| 6.782033406957482|
|   GSO|  7.02220425046734|
|   MYR| 5.589231543912653|
|   ISO|   5.7409200968523|
+------+------------------+
only showing top 20 rows



**3. What day the delays are the worst?**

In [29]:
dataFrame.createOrReplaceTempView("AIRLINESINFO")
spark.sql("select DayOfWeek, avg(ArrDelay) as avg_delay from AIRLINESINFO " +
          "group by DayOfWeek order by avg_delay desc").show(1)

+---------+-----------------+
|DayOfWeek|        avg_delay|
+---------+-----------------+
|        5|9.606953425895007|
+---------+-----------------+
only showing top 1 row



**4. What is the average departure delay from each airport?**

In [30]:
# This is same as question #2. Find the mean departure delay per origination airport.
delayDF_cache.show(20)

+------+------------------+
|Origin|     avg(DepDelay)|
+------+------------------+
|   BGM| 4.103673055154378|
|   DLG|             10.75|
|   PSE|1.9544658493870404|
|   INL| 6.168539325842697|
|   MSY| 6.554481654634312|
|   GEG| 6.038673098079038|
|   SNA| 5.813108982130751|
|   BUR| 6.695134420234224|
|   GRB| 4.685745396013824|
|   GTF|3.1319266055045873|
|   IDA| 3.023048836147878|
|   GRR| 5.625893559603069|
|   LWB| 7.619393939393939|
|   PVU|               0.0|
|   EUG| 7.953983363108531|
|   PSG|12.997746320163392|
|   PVD| 6.782033406957482|
|   GSO|  7.02220425046734|
|   MYR| 5.589231543912653|
|   ISO|   5.7409200968523|
+------+------------------+
only showing top 20 rows



**5. Which day of the week is the most of the flights cancelled?**

In [31]:
from pyspark.sql.functions import sum, desc
dataFrame.groupBy(['DayOfWeek']).agg(sum('Cancelled').alias('sum_cancelled')).sort(desc('sum_cancelled')).show(1)

+---------+-------------+
|DayOfWeek|sum_cancelled|
+---------+-------------+
|        2|       376554|
+---------+-------------+
only showing top 1 row



**6. Which day of the month is the most of the flights cancelled?**

In [32]:
from pyspark.sql.functions import sum, desc
dataFrame.groupBy(['DayofMonth']).agg(sum('Cancelled').alias('sum_cancelled')).sort(desc('sum_cancelled')).show(1)

+----------+-------------+
|DayofMonth|sum_cancelled|
+----------+-------------+
|        11|        99407|
+----------+-------------+
only showing top 1 row



**7. Find the on-time (ArrTime - CRSArrTime <= 0) performance for each unique carrier.**

In [9]:
from pyspark.sql.functions import when, count, col, lit, desc
perfDF = dataFrame.groupBy(['UniqueCarrier']).agg((count(lit(when(col('ArrDelay') <= 0, 1)))/count(lit(1))).alias('performance')).sort(desc('performance'))
perfDF.show(20)

+-------------+------------------+
|UniqueCarrier|       performance|
+-------------+------------------+
|           HA| 0.745585473902977|
|           AQ|0.6094985781929123|
|           9E|0.5894380482824402|
|           DH|0.5885502714823093|
|           TZ|0.5800067172056425|
|           OO| 0.575008581773381|
|           WN|0.5678857978538087|
|       ML (1)|0.5605618645747784|
|           B6|0.5397212762574553|
|           YV|0.5394318405350469|
|           OH|0.5393689010064364|
|           XE|0.5391878259411848|
|           FL|0.5381246946973374|
|           NW|0.5351327702830385|
|           MQ|0.5347082539485878|
|           AA|0.5262442285093536|
|           F9|0.5157259955246648|
|           CO|0.5153851290998489|
|       PA (1)|0.5112203360882065|
|           EV|0.5094133063708334|
+-------------+------------------+
only showing top 20 rows

